In [1]:
from pystac import Item
from odc.stac import load
import s3fs
from ipyleaflet import basemaps
import odc.geo.xr  # noqa: F401

In [ ]:
tiles = [
    ("047", "238"),
    ("047", "239"),
    ("048", "238"),
    ("048", "239"),
]

items = []

for tile in tiles:
    url = f"https://data.ldn.auspatious.com/geo_ls_lp/0_1_0/{tile[0]}/{tile[1]}/2023/geo_ls_lp_2023_{tile[0]}_{tile[1]}.stac-item.json"
    items.append(Item.from_file(url))

In [ ]:
path = "s3://data.ldn.auspatious.com/geo_ls_lp/**/*.stac-item.json"

fs = s3fs.S3FileSystem(anon=True)
stac_keys = fs.glob(path)

items = [Item.from_file(f"https://{key}") for key in stac_keys]

print(f"Found {len(items)} items")

In [ ]:
# # Bounding box covering only viti levu
# bbox = [177.20, -18.30, 178.75, -17.25]

# Bounding box covering Belize
bbox = [-89.5, 15.8, -88.1, 18.5]

# # Bounding box covering Trinidad and Tobago
# bbox = [-62.0, 10.0, -60.5, 11.5]

data = load(
    items,
    chunks={"time": 1, "x": 2500, "y": 2500},
    bbox=bbox,
).squeeze().compute()

data

In [ ]:
data.evi2.odc.explore(tiles=basemaps.Esri.WorldImagery)

https://www.fused.io/server/v1/realtime-shared/fsh_5vRfred1ROpHbAPRZ0sqIE/run/tiles/13/2711/3764?dtype_out_raster=png&year=2023&variable=evi2

In [23]:
m.save("viti_levu_all.html")

In [ ]:
import ipyleaflet

m = ipyleaflet.Map(
    center=(-18.145715165279427, 178.42568862657757),
    zoom=10.093458956904048,
    basemap=ipyleaflet.basemaps.Esri.WorldImagery,
)
l = ipyleaflet.TileLayer(
    url=f"https://www.fused.io/server/v1/realtime-shared/fsh_5vRfred1ROpHbAPRZ0sqIE/run/tiles/{{z}}/{{x}}/{{y}}",
    tile_size=512,
    zoom_offset=-1,
    cross_origin=True,
    show_loading=True,
    name="test"
)
m.add_layer(l)

m.add_control(ipyleaflet.LayersControl(position="topright"))

m